TO DO:
- version w/o lemmatizing?
- reusable item for importing into pipeline 

In [2]:
#imports and set your path 
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import PlaintextCorpusReader, wordnet
path = '/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/'

Part 1: 
- extract needed info from MJI and musoW dumps 
- format, align and remove dupes
- bounce to text  

In [2]:
#read mji csv and turn into two column df for names and desc 
df_mji = pd.read_csv(path+'MJI/MJI_data.csv', keep_default_na=False, dtype='string')
df_mji_small = df_mji.iloc[:, [0, 5]].copy()
df_mji_small['Title'] = df_mji_small['Title'].str.lower().str.strip()
df_mji_small['Description'] = df_mji_small['Description'].str.lower().str.strip()

In [3]:
negative = pd.read_csv(path+'negative_set.csv', keep_default_na=False, dtype='string')
negative['Title'] = negative['Title'].str.lower().str.strip()
negative['Description'] = negative['Description'].str.lower().str.strip()

In [3]:
#read musow json dump and turn into df w/ same columns 
with open(path+'MUSOW/musow_name_desc_url_cat.json') as file:
    data = json.load(file)
    
musow_names = [result['name']['value'].strip().lower() for result in data['results']['bindings']]
musow_desc = [result['description']['value'].strip().lower() for result in data['results']['bindings']]
df_musow = pd.DataFrame(columns=['Title', 'Description'])
df_musow['Title'] = musow_names
df_musow['Description'] = musow_desc
df_musow = df_musow.astype('string')
df_musow.to_pickle(path+'KEYWORDS/musoW_keywords.pkl')

In [4]:
#remove duplicates from MJI set based on title field 
df_mji_small[~df_mji_small['Title'].isin(df_musow['Title'])].dropna()
df_mji_small.to_pickle(path+'KEYWORDS/MJI_keywords.pkl')

In [5]:
#save each df to a single text file for processing 
with open(path+'KEYWORDS/mji_corpus.txt', 'a') as f:
    dfAsString = df_mji_small.to_string(header=False, index=False)
    f.write(dfAsString)
with open(path+'KEYWORDS/musow_corpus.txt', 'a') as f:
    dfAsString = df_musow.to_string(header=False, index=False)
    f.write(dfAsString)

In [5]:
with open(path+'KEYWORDS/negative.txt', 'a') as f:
    dfAsString = negative.to_string(header=False, index=False)
    f.write(dfAsString)

Part 2:
- text processing using nltk: remove punctuations, tokenize, remove nltk stopwords + custom list, lemmatize 
- get top 35 keywords 
- get bigrams in which each word is in the top 35 kws 

In [6]:
#set corpus variables + vars for cleaning 
mji_corpus = PlaintextCorpusReader(path+'KEYWORDS/mji_corpus.txt', '.*\.txt')
musow_corpus = PlaintextCorpusReader(path+'KEYWORDS/musow_corpus.txt', '.*\.txt')
negative_corpus = PlaintextCorpusReader(path+'KEYWORDS/negative.txt', '.*\.txt')
stopwords = nltk.corpus.stopwords.words('english')
punct_tokenizer = nltk.RegexpTokenizer(r"\w+")
custom_stopwords = ['available', '000', 'including', 'also', 'includes', 'website', 'new', 'include', 'well', 'based', 'source', 'sources', 'contains', 'search']

In [7]:
#function to extract top 35 keywords and return them alongside bigrams that include two words within top 35
def keywords(input_corpus):
    string = input_corpus.raw('')
    tokenised = punct_tokenizer.tokenize(string)
    clean = [w for w in tokenised if w not in stopwords]
    clean_2 = [w for w in clean if w not in custom_stopwords]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w) for w in clean_2]
    freqdist_lem = nltk.FreqDist(lemmatized)
    most_common_lem = freqdist_lem.most_common(35)
    most_common_lem_list = []
    for t in most_common_lem:
        most_common_lem_list.append(t[0])
    bigrams = nltk.bigrams(lemmatized)
    freqdist_bg = nltk.FreqDist(bigrams)
    search_bigrams = []
    for k, v in freqdist_bg.items():
        if k[0] in most_common_lem_list:
            if k[1] in most_common_lem_list:
                if v > 5:
                    k = ' '.join(k)
                    search_bigrams.append([k, v])
    kw_pd = pd.DataFrame(columns=['Most Common KW', 'KW Freq', 'Bigrams', 'Bigram Freq'])
    kw_pd['Most Common KW'] = pd.Series([w[0] for w in most_common_lem])
    kw_pd['KW Freq'] = pd.Series([w[1] for w in most_common_lem])
    kw_pd['Bigrams'] = pd.Series([b[0] for b in search_bigrams])
    kw_pd['Bigram Freq'] = pd.Series([b[1] for b in search_bigrams])
    return kw_pd

In [8]:
#process each and pickle 
musowkw = keywords(musow_corpus)
musowkw.to_pickle(path+'KEYWORDS/musowkw.pkl')
mjikw = keywords(mji_corpus)
mjikw.to_pickle(path+'KEYWORDS/mjikw.pkl')


In [8]:
negativekw = keywords(negative_corpus)

In [ ]:
negativekw

Part 3:
- Analyze results, differences 
- Create reusable item for needed results 

In [14]:
musowkw['Source'] = 'musow'
mjikw['Source'] = 'mji'
different = pd.concat([musowkw, mjikw]).drop_duplicates(keep=False)

In [20]:
different_kw = different.copy()
different_kw = different_kw[['Most Common KW', 'KW Freq', 'Source']]
different_kw = different_kw.sort_values(by='KW Freq', ascending=False)
different_kw.to_csv(path+'KEYWORDS/kw_summary.csv')
different_bg = different.copy()
different_bg = different_bg[['Bigrams', 'Bigram Freq', 'Source']]
different_bg = different_bg.sort_values(by='Bigram Freq', ascending=False)
different_bg.to_csv(path+'KEYWORDS/bg_summary.csv')